# Weekly Project 5 
## Implementation of global registration 
### Task 1
Today, your task is to implement a global registration algorithm.

It should be able to roughly align two point clouds.
Implement the global registration, and then try the following:

In [1]:
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

1. Can you fit `r1.pcd` and `r2.pcd`?

In [3]:
path = "C:/Users/shaia/Documents/Opgaveregning/AutoSys/4. semester/Perception for autonome systemer/Projekt/Projekt 5/"

source = o3d.io.read_point_cloud(path + "global_registration/r1.pcd")
target = o3d.io.read_point_cloud(path + "global_registration/r2.pcd")

draw_registrations(source, target, recolor=True)

voxel_size = 0.05

source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

source_down.estimate_normals()
target_down.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

point_to_point = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane = o3d.pipelines.registration.TransformationEstimationPointToPlane()
distance_threshold = voxel_size * 1.5

ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, 
    source_fpfh, target_fpfh, 
    True, distance_threshold,
    point_to_point)
draw_registrations(source, target, ransac_result.transformation, True)

2. Can you fit `car1.ply` and `car2.ply`?
The corresponding files are in the `global_registration` folder.

In [4]:
source = o3d.io.read_point_cloud(path + "global_registration/car1.ply")
target = o3d.io.read_point_cloud(path + "global_registration/car2.ply")

draw_registrations(source, target, recolor=True)

voxel_size = 0.05

source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

source_down.estimate_normals()
target_down.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

point_to_point = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane = o3d.pipelines.registration.TransformationEstimationPointToPlane()
distance_threshold = voxel_size * 1.5

ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, 
    source_fpfh, target_fpfh, 
    True, distance_threshold,
    point_to_point)
draw_registrations(source, target, ransac_result.transformation, True)

### Task 2 (Challange)
Implement local registration.

In [5]:
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

threshold = 0.02

print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, ransac_result.transformation)
print(evaluation)

point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
icp_result = o3d.pipelines.registration.registration_icp(
    source, target, threshold, ransac_result.transformation, point_to_plane)

draw_registrations(source, target, icp_result.transformation, True)

Initial alignment
RegistrationResult with fitness=9.742326e-01, inlier_rmse=8.672192e-03, and correspondence_set size of 161935
Access transformation to get result.
